# Day 02 — Baseline model (logistic regression)

I’m using a tiny synthetic dataset just to keep the workflow clear.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

data = {
    "study_hours": [1, 2, 3, 4, 5, 6, 7, 8],
    "practice_tests": [0, 1, 0, 1, 2, 2, 3, 3],
    "passed": [0, 0, 0, 1, 1, 1, 1, 1],
}
df = pd.DataFrame(data)
df.head()

In [ ]:
X = df[["study_hours", "practice_tests"]]
y = df["passed"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_test)
accuracy_score(y_test, preds)

In [ ]:
cm = confusion_matrix(y_test, preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Baseline confusion matrix")
plt.show()